# OSMnx + igraph

Author: [Geoff Boeing](https://geoffboeing.com/)

First install [igraph](https://igraph.org/python/) or run Jupyter from the [Docker container](https://github.com/gboeing/osmnx/tree/master/docker) (which already has it installed along with OSMnx and NetworkX).

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
pip install igraph

     |████████████████████████████████| 119 kB 5.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:

import igraph as ig
import networkx as nx
import numpy as np
import operator
import osmnx as ox

ox.config(use_cache=True, log_console=True)
weight = 'length'

ox.__version__

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

## Construct graphs

In [ ]:
# create networkx graph
G_nx = ox.graph_from_place('Piedmont, CA, USA', network_type='drive')
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

In [ ]:
%%time
# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(list(G_nx.nodes()))
G_ig.add_edges(list(G_nx.edges()))
G_ig.vs['osmid'] = list(nx.get_node_attributes(G_nx, 'osmid').values())
G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

In [ ]:
assert len(G_nx.nodes()) == G_ig.vcount()
assert len(G_nx.edges()) == G_ig.ecount()

## Shortest paths

In [ ]:
source = list(G_nx.nodes())[0]
target = list(G_nx.nodes())[-1]

In [ ]:
%%time
path1 = G_ig.get_shortest_paths(v=source, to=target, weights=weight)[0]

In [ ]:
%%time
path2 = nx.shortest_path(G_nx, source, target, weight=weight)

In [ ]:
assert path1 == path2

In [ ]:
%%time
path_length1 = G_ig.shortest_paths(source=source, target=target, weights=weight)[0][0]

In [ ]:
%%time
path_length2 = nx.shortest_path_length(G_nx, source, target, weight)

In [ ]:
assert path_length1 == path_length2

## Centrality analysis

In [ ]:
%%time
closeness1 = G_ig.closeness(vertices=None, mode='ALL', cutoff=None, weights=weight, normalized=True)
max_closeness1 = np.argmax(closeness1)

In [ ]:
%%time
closeness2 = nx.closeness_centrality(G_nx, distance=weight, wf_improved=True)
max_closeness2 = max(closeness2.items(), key=operator.itemgetter(1))[0]

In [ ]:
assert G_nx.nodes[max_closeness2]['osmid'] == G_ig.vs[max_closeness1]['osmid']